In [2]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
from glob import glob


def parse_energy_file(file_path):
    metrics = {}
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                parts = line.split()
                if len(parts) < 2:
                    continue
                
                full_metric = parts[0]
                value = float(parts[-1])
                
                if "{" in full_metric and "}" in full_metric:
                    metric_name = full_metric.split("{")[0]
                    labels = full_metric.split("{")[1].split("}")[0]
                    
                    if "mode=" in labels:
                        mode = labels.split("mode=")[1].split(",")[0].strip('"')
                        key = f"{metric_name}_{mode}"
                    else:
                        key = metric_name
                else:
                    key = full_metric
                
                metrics[key] = value
            except (IndexError, ValueError) as e:
                print(f"Error parsing line '{line}': {str(e)}")
                continue
    return metrics


def calculate_energy_consumption(before_metrics, after_metrics):
    if not before_metrics or not after_metrics:
        return None

    total_joules_key = "kepler_container_joules_total_dynamic"

    dynamic_key = f'{total_joules_key}'
    if dynamic_key in after_metrics and dynamic_key in before_metrics:
        total_energy = after_metrics[dynamic_key] - before_metrics[dynamic_key]
    else:
        return None

    return total_energy if total_energy > 0 else None


def process_test_directory(base_dir, test_name, requests, gateway=False):
    """Process all report directories for a specific test case."""
    energy_consumptions = []

    # Determine the path pattern based on whether it's gateway or not
    path_pattern = os.path.join(base_dir, str(requests), "report_*")

    # Process each report directory
    for report_dir in glob(path_pattern):
        try:
            before_file_gateway = None
            after_file_gateway = None
            before_metrics_gateway = None
            after_metrics_gateway = None
            energy_gateway = 0
            
            if gateway:
                before_file_gateway = os.path.join(report_dir, "before_gateway.txt")
                after_file_gateway = os.path.join(report_dir, "after_gateway.txt")
                
            before_file = os.path.join(report_dir, "before.txt")
            after_file = os.path.join(report_dir, "after.txt")

            if os.path.exists(before_file) and os.path.exists(after_file):
                if gateway and os.path.exists(before_file_gateway) and os.path.exists(after_file_gateway):
                    before_metrics_gateway = parse_energy_file(before_file_gateway)
                    after_metrics_gateway = parse_energy_file(after_file_gateway)
                    energy_gateway = calculate_energy_consumption(
                        before_metrics_gateway, after_metrics_gateway
                    )
                
                
                before_metrics = parse_energy_file(before_file)
                after_metrics = parse_energy_file(after_file)
                energy = calculate_energy_consumption(before_metrics, after_metrics)
                if energy is not None:
                    energy_consumptions.append(energy + energy_gateway)
        except Exception as e:
            print(f"Error processing {report_dir}: {str(e)}")
            continue

    if not energy_consumptions:
        return None
    # Calculate statistics
    stats = {
        "Test": test_name,
        "Requests": requests,
        "Mean": round(np.mean(energy_consumptions), 2),
        "Med": round(np.median(energy_consumptions), 2),
        "Min": round(np.min(energy_consumptions), 2),
        "Max": round(np.max(energy_consumptions), 2),
        "Std": round(np.std(energy_consumptions), 2),
    }

    return stats


def get_endpoint_name(path):
    """Extract endpoint name from path"""
    parts = path.split("/")
    # Look for parts like 'json_get_user_id' or 'protobuf_get_all_users'
    for part in parts:
        if part.startswith("json_") or part.startswith("protobuf_"):
            return part.replace("json_", "").replace("protobuf_", "")
    return "unknown_endpoint"


def custom_sort(row):
    """Custom sorting function for the desired order"""
    test = row["Test"]
    requests = row["Requests"]

    if "Gateway" not in test:
        if requests == 1000:
            return 0 if "JSON" in test else 1
        else:
            return 2 if "JSON" in test else 3
    else:
        if requests == 1000:
            return 4 if "JSON" in test else 5
        else:
            return 6 if "JSON" in test else 7



test_dirs = [
    "../Control_Project_JSON/k6/json_get_all_products_entities/reports/json_get_all_products_entities",
    "../Control_Project_JSON/k6/json_get_user_id/reports/json_get_user_id",
    "../Control_Project_JSON/k6/json_get_all_users/reports/json_get_all_users",
    "../Control_Project_JSON/k6/json_create_user/reports/json_create_user",
    "../Control_Project_JSON/k6/json_update_user/reports/json_update_user",
    "../Control_Project_JSON/k6/json_delete_user/reports/json_delete_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_users/reports/protobuf_get_all_users",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_user_id/reports/protobuf_get_user_id",
    "../Experimental_Group_ProtoBuff/k6/protobuf_create_user/reports/protobuf_create_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_update_user/reports/protobuf_update_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_delete_user/reports/protobuf_delete_user",
    "../Gateway/k6_json/json_get_all_products_entities/reports/json_get_all_products_entities_gateway",
    "../Gateway/k6_json/json_get_all_users/reports/json_get_all_users_gateway",
    "../Gateway/k6_json/json_get_user_id/reports/json_get_user_id_gateway",
    "../Gateway/k6_json/json_create_user/reports/json_create_user_gateway",
    "../Gateway/k6_json/json_update_user/reports/json_update_user_gateway",
    "../Gateway/k6_json/json_delete_user/reports/json_delete_user_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_users/reports/protobuf_get_all_users_gateway",
    "../Gateway/k6_protobuf/protobuf_get_user_id/reports/protobuf_get_user_id_gateway",
    "../Gateway/k6_protobuf/protobuf_create_user/reports/protobuf_create_user_gateway",
    "../Gateway/k6_protobuf/protobuf_update_user/reports/protobuf_update_user_gateway",
    "../Gateway/k6_protobuf/protobuf_delete_user/reports/protobuf_delete_user_gateway",
]

# Group by endpoint
endpoint_data = {}

for test_dir in test_dirs:
    if not os.path.exists(test_dir):
        print(f"Directory not found: {test_dir}")
        continue

    # Determine endpoint name, test type, and whether it's gateway
    endpoint = get_endpoint_name(test_dir)
    is_gateway = "Gateway" in test_dir
    is_protobuf = "protobuf" in test_dir.lower()

    test_type = "PROTOBUF" if is_protobuf else "JSON"
    if is_gateway:
        test_type += " + Gateway"

    # Process for both request counts
    for requests in [1000, 10000]:
        stats = process_test_directory(test_dir, test_type, requests, is_gateway)
        if stats:
            if endpoint not in endpoint_data:
                endpoint_data[endpoint] = []
            endpoint_data[endpoint].append(stats)

# Print results for each endpoint
for endpoint, results in endpoint_data.items():
    df = pd.DataFrame(results)

    # Add sorting key and sort
    df["sort_key"] = df.apply(custom_sort, axis=1)
    df = df.sort_values("sort_key").drop("sort_key", axis=1)

    df = df[["Test", "Requests", "Mean", "Med", "Min", "Max", "Std"]]

    print(
        f"\nEnergy Consumption Statistics for {endpoint.replace('_', ' ').title()} (Joules):"
    )
    display(df)

    comparisons = []
    for (is_gateway, req_count), group in df.groupby([df["Test"].str.contains("Gateway"), "Requests"]):
        if len(group) == 2:  
            json_row = group[group["Test"].str.contains("JSON")].iloc[0]
            proto_row = group[group["Test"].str.contains("PROTOBUF")].iloc[0]

            test_name = "PROTOBUF + Gateway vs JSON + Gateway" if is_gateway else "PROTOBUF vs JSON"
            diff_pcts = {
                "Test": f"{test_name} ({req_count} requests)",
                "Mean %": (
                    (json_row["Mean"] - proto_row["Mean"]) / json_row["Mean"]
                )
                * 100,
                "Median %": ((json_row["Med"] - proto_row["Med"]) / json_row["Med"])
                * 100,
                "Min %": ((json_row["Min"] - proto_row["Min"]) / json_row["Min"])
                * 100,  
                "Max %": ((json_row["Max"] - proto_row["Max"]) / json_row["Max"])
                * 100,
                "Std %": ((json_row["Std"] - proto_row["Std"]) / json_row["Std"])
                * 100,
            }
            comparisons.append(diff_pcts)

    # Create comparison table
    comparison_df = pd.DataFrame(comparisons)
    comparison_df = comparison_df.round(2)  # Round to 2 decimal places

    print(f"\nPercentage Comparison for {endpoint.replace('_', ' ').title()}:")
    display(comparison_df)


Energy Consumption Statistics for Get All Products Entities (Joules):


,Test,Requests,Mean,Med,Min,Max,Std
0,JSON,1000,32.84,23.11,10.50,79.98,24.89
2,PROTOBUF,1000,25.62,16.91,10.91,56.53,16.58
1,JSON,10000,90.87,84.76,71.82,129.83,20.12
3,PROTOBUF,10000,83.84,74.89,69.55,120.64,18.76
4,JSON + Gateway,1000,74.82,45.34,38.23,182.71,54.94
6,PROTOBUF + Gateway,1000,38.87,31.19,22.31,74.71,18.59
5,JSON + Gateway,10000,245.09,227.95,217.84,327.93,41.65
7,PROTOBUF + Gateway,10000,223.20,213.54,202.86,262.35,21.17



Percentage Comparison for Get All Products Entities:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),21.99,26.83,-3.90,29.32,33.39
1,PROTOBUF vs JSON (10000 requests),7.74,11.64,3.16,7.08,6.76
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,48.05,31.21,41.64,59.11,66.16
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,8.93,6.32,6.88,20.00,49.17



Energy Consumption Statistics for Get User Id (Joules):


...

,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),-0.84,3.18,1.98,-9.41,-27.94
1,PROTOBUF vs JSON (10000 requests),-3.51,-3.05,-0.05,-5.15,-67.83
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,2.21,5.00,-2.39,-3.55,-13.45
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,-1.17,-1.18,-4.16,1.59,54.05



Energy Consumption Statistics for Delete User (Joules):


,Test,Requests,Mean,Med,Min,Max,Std
0,JSON,1000,35.65,36.87,27.40,39.37,4.23
2,PROTOBUF,1000,39.37,39.01,37.40,42.79,1.81
1,JSON,10000,230.78,235.65,114.00,342.69,81.48
3,PROTOBUF,10000,241.09,257.89,98.06,366.44,94.25
4,JSON + Gateway,1000,54.30,50.67,49.19,62.52,5.67
6,PROTOBUF + Gateway,1000,54.82,55.76,42.40,64.06,6.99
5,JSON + Gateway,10000,426.92,431.64,281.74,544.79,90.87
7,PROTOBUF + Gateway,10000,419.44,438.79,270.96,542.62,96.92



Percentage Comparison for Delete User:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),-10.43,-5.80,-36.50,-8.69,57.21
1,PROTOBUF vs JSON (10000 requests),-4.47,-9.44,13.98,-6.93,-15.67
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,-0.96,-10.05,13.80,-2.46,-23.28
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,1.75,-1.66,3.83,0.40,-6.66
